In [2]:
using Random
using LinearAlgebra

function U(x::Vector{Float64}, m::Vector{Int64})
    N = length(x)
    M = Matrix{ComplexF64}(I, N, N)
    for k in 1:N
        if k > 1
            h = x[k] - x[1:k-1]
            h /= norm(h)
            for j in 1:k-1
                u = exp(im * 2π * h' * [0:N-1;] / N)
                M[j, k] = u[m[j]] / u[m[k]]
                M[k, j] = conj(M[j, k])
            end
        end
        u = exp.(im * 2π * x[k] * [0:N-1;] / N)
        M[k, k] = u[m[k]]
    end
    return M
end

function markov_chain_sampling(x::Vector{Float64}, m::Vector{Int64}, δx::Float64, Mcond::Int64)
    N = length(x)
    x_new = copy(x)
    for i in 1:N
        h = x_new[i] - x[1:i-1]
        h /= norm(h)
        U_new = U(x_new, m)
        U_old = U(x, m)
        for j in 1:Mcond
            x_trial = x_new[i] + δx * randn()
            h_trial = x_trial - x[1:i-1]
            h_trial /= norm(h_trial)
            U_trial = U([x[1:i-1]; x_trial; x[i+1:N]], m)
            P = abs2(dot(U_trial[:, i], h_trial) / dot(U_old[:, i], h))
            if rand() < P
                x_new[i] = x_trial
                U_old = copy(U_trial)
            end
        end
    end
    return x_new
end

function FFS_continuous(N::Int64, Mcond::Int64, δx::Float64)
    m = randperm(N)
    x = rand(N)
    for i in 1:N
        x = markov_chain_sampling(x, m, δx, Mcond)
    end
    return x
end

FFS_continuous(10, 10, 0.1)


LoadError: MethodError: no method matching -(::Float64, ::Vector{Float64})
For element-wise subtraction, use broadcasting with dot syntax: scalar .- array
[0mClosest candidates are:
[0m  -(::Union{Float16, Float32, Float64}) at float.jl:381
[0m  -(::T, [91m::T[39m) where T<:Union{Float16, Float32, Float64} at float.jl:384
[0m  -(::Union{Float16, Float32, Float64}, [91m::BigFloat[39m) at mpfr.jl:465
[0m  ...